In [1]:
import requests
import pandas as pd
import os

def fetch_data(url, headers):
    """Fetch data from Yahoo Finance API."""
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return {}

def get_stock_data(symbols, api_key):
    """Fetch basic stock information for given symbols."""
    headers = {"x-api-key": api_key}
    url = f"https://yfapi.net/v6/finance/quote?symbols={','.join(symbols)}"
    response = fetch_data(url, headers)

    stock_data = []
    for symbol in symbols:
        quote = next((item for item in response.get("quoteResponse", {}).get("result", []) if item.get('symbol') == symbol), {})
        stock_data.append({
            'Stock Ticker': symbol,
            'Company': quote.get('longName', 'Unavailable'),
            'Current Market Price': quote.get('regularMarketPrice', 'Unavailable')
        })

    return pd.DataFrame(stock_data)

def get_additional_data(symbol, api_key):
    """Fetch additional financial data, explicitly showing 52-Week High/Low and ROA."""
    url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}?modules=summaryDetail,financialData"
    headers = {"x-api-key": api_key}
    response = fetch_data(url, headers)
    summary = response.get("quoteSummary", {}).get("result", [{}])[0]

    data = {
        '52-Week High': summary.get("summaryDetail", {}).get("fiftyTwoWeekHigh", {}).get("raw", 'Unavailable'),
        '52-Week Low': summary.get("summaryDetail", {}).get("fiftyTwoWeekLow", {}).get("raw", 'Unavailable'),
        'Return on Assets (ROA)': summary.get("financialData", {}).get("returnOnAssets", {}).get("raw", 'Unavailable')
    }

    return pd.DataFrame([data])

def main():
    """Main function to interactively get stock market data."""
    api_key = os.getenv("YF_API_KEY") or input("Enter your Yahoo Finance API key: ").strip()

    # Ensure the user enters at least two stock symbols
    while True:
        symbols = input("Enter at least two stock symbols separated by commas: ").strip().split(',')
        symbols = [s.strip().upper() for s in symbols if s.strip()]
        if len(symbols) >= 2:
            break
        print("Please enter at least two stock symbols.")

    # Fetch and display stock data
    stock_df = get_stock_data(symbols, api_key)
    print("\nStock Data:")
    print(stock_df.to_string(index=False))  # Clean display

    # Fetch additional stock data for a single symbol
    symbol = input("\nEnter a stock symbol to fetch 52-Week High/Low & ROA data: ").strip().upper()
    additional_df = get_additional_data(symbol, api_key)

    print("\nAdditional Data (52-Week High, 52-Week Low, ROA):")
    print(additional_df.to_string(index=False))

if __name__ == "__main__":
    main()


Enter your Yahoo Finance API key: gqHkm1vwB68rrYFYvJQMPaTqlFilQlTq9lIns7hf
Enter at least two stock symbols separated by commas: AAPL, MSFT

Stock Data:
Stock Ticker               Company  Current Market Price
        AAPL            Apple Inc.                240.36
        MSFT Microsoft Corporation                399.73

Enter a stock symbol to fetch 52-Week High/Low & ROA data: AAPL

Additional Data (52-Week High, 52-Week Low, ROA):
 52-Week High  52-Week Low  Return on Assets (ROA)
        260.1       164.08                 0.22519
